# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

## Part 1 - Data Preprocessing

### Importing the dataset

In [2]:
dataset = pd.read_excel('End_time.xlsx')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Charge_mass        576 non-null    float64
 1   Standoff_distance  576 non-null    int64  
 2   End_time           576 non-null    float64
 3   End_index          576 non-null    int64  
dtypes: float64(2), int64(2)
memory usage: 18.1 KB


In [3]:
y = dataset['End_time']
X = dataset.drop(['End_time','End_index'], axis=1)
print(X.shape, y.shape)

(576, 2) (576,)


In [4]:
# convert to numpy array
X = np.array(X)
y = np.array(y)

### Splitting the dataset into the Training set and Test set

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

### Hyperparameter tuning - layers, neurons, activation function

In [6]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor as KR
import math
def FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes):
    layers = []
    
    nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
    nodes = first_layer_nodes
    for i in range(1, n_layers+1):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    
    return layers

In [7]:
from tensorflow.keras.optimizers import Adam
def create_model(n_layers, first_layer_nodes, last_layer_nodes, activation_func):
    model = Sequential()
    n_nodes = FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes)
    for i in range(1, n_layers):
        if i==1:
            model.add(Dense(units = first_layer_nodes,  input_shape=(X_train.shape[1],), activation=activation_func))
        else:
            model.add(Dense(n_nodes[i-1], activation=activation_func))
            
    model.add(Dense(1, activation='linear'))
    opt = Adam(learning_rate=0.001)
    model.compile(optimizer = opt, loss = 'mean_squared_error', metrics = ['mae'])
    return model

##Wrap model into scikit-learn
model = KR(build_fn=create_model, epochs = 500, batch_size = 50)

C:\Users\Chathura Gamage\AppData\Local\Temp\ipykernel_14064\3503773462.py:17: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KR(build_fn=create_model, epochs = 500, batch_size = 50)


In [8]:
activation_funcs = ['relu', 'softplus', 'leaky_relu'] 
param_grid = dict(n_layers=[2,3],
                  first_layer_nodes = [5, 6, 7, 8],
                  last_layer_nodes = [2,3,4,5],
                  activation_func = activation_funcs)
grid = GridSearchCV(estimator = model, param_grid = param_grid,n_jobs=-1, cv=5, scoring = 'r2')
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/500
9/9 [==============================] - 3s 3ms/step - loss: 47.8066 - mae: 5.9531
Epoch 2/500
9/9 [==============================] - 0s 2ms/step - loss: 39.5738 - mae: 5.3699
Epoch 3/500
9/9 [==============================] - 0s 2ms/step - loss: 32.4967 - mae: 4.8389
Epoch 4/500
9/9 [==============================] - 0s 2ms/step - loss: 26.4340 - mae: 4.3529
Epoch 5/500
9/9 [==============================] - 0s 2ms/step - loss: 20.9728 - mae: 3.8875
Epoch 6/500
9/9 [==============================] - 0s 2ms/step - loss: 15.8137 - mae: 3.3674
Epoch 7/500
9/9 [==============================] - 0s 2ms/step - loss: 11.1898 - mae: 2.8173
Epoch 8/500
9/9 [==============================] - 0s 2ms/step - loss: 7.1417 - mae: 2.2499
Epoch 9/500
9/9 [==============================] - 0s 2ms/step - loss: 4.2074 - mae: 1.7145
Epoch 10/500
9/9 [==============================] - 0s 2ms/step - loss: 2.3138 - mae: 1.2475
Epoch 11/500
9/9 [==============================] - 0s 2ms/step - loss: 

### Hyperparameter tuning - batch size, epoch, optimizer, learning rate

In [9]:
# Function to create model, required for KerasRegressor
def create_model(optimizer):
 # create model
 model = Sequential()
 model.add(Dense(units=5, input_shape=(X_train.shape[1],), activation='leaky_relu'))
 model.add(Dense(units=5, activation='leaky_relu'))
 model.add(Dense(units=5, activation='leaky_relu'))
 model.add(Dense(units=1, activation='linear'))
 # Compile model
 model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['mae'])
 return model

In [10]:
from scikeras.wrappers import KerasRegressor
# create model
model = KerasRegressor(model=create_model)

# define the grid search parameters
batch_size = [30,40,50]
epochs = [300, 400, 500]
optimizer = ['RMSprop', 'Adam', 'Adamax', 'Nadam']
learn_rate = [0.001, 0.01, 0.1]

# gridsearch
param_grid = dict(batch_size=batch_size, epochs=epochs, model__optimizer=optimizer, optimizer__learning_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5, scoring = 'r2')
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/500
8/8 [==============================] - 0s 2ms/step - loss: 64.2253 - mae: 7.1136
Epoch 2/500
8/8 [==============================] - 0s 1ms/step - loss: 50.8843 - mae: 6.3209
Epoch 3/500
8/8 [==============================] - 0s 2ms/step - loss: 40.3574 - mae: 5.6101
Epoch 4/500
8/8 [==============================] - 0s 2ms/step - loss: 31.8090 - mae: 4.9817
Epoch 5/500
8/8 [==============================] - 0s 1ms/step - loss: 25.3351 - mae: 4.4276
Epoch 6/500
8/8 [==============================] - 0s 2ms/step - loss: 19.8669 - mae: 3.8940
Epoch 7/500
8/8 [==============================] - 0s 2ms/step - loss: 14.3347 - mae: 3.2812
Epoch 8/500
8/8 [==============================] - 0s 1ms/step - loss: 9.0758 - mae: 2.5909
Epoch 9/500
8/8 [==============================] - 0s 1ms/step - loss: 5.4446 - mae: 1.9566
Epoch 10/500
8/8 [==============================] - 0s 2ms/step - loss: 3.0270 - mae: 1.4007
Epoch 11/500
8/8 [==============================] - 0s 2ms/step - loss: 